In [1]:
from pytorch_tabnet.tab_model import TabNetClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
import torch
from sklearn.model_selection import StratifiedKFold

In [ ]:
df=pd.read_csv('../balanced_cleaned_trian_dataset.csv',index_col=0)
test_set=pd.read_csv('../testset.csv',index_col=0)

grouped = df.groupby("smoking")
df_0 = grouped.get_group(0)
df_1 = grouped.get_group(1)
df_0_half = df_0.sample(frac=0.5, random_state=42)
df_1_half = df_1.sample(frac=0.5, random_state=42)
df_half = pd.concat([df_0_half, df_1_half])

df_half = df_half.sample(frac=1, random_state=42).reset_index(drop=True)

In [3]:
train_copy=df_half.drop('id',axis=1)
X = train_copy.drop('smoking', axis=1)
y = train_copy['smoking']

test_set=test_set.drop('id',axis=1)
X_test_new= test_set.drop('smoking', axis=1)
y_test_new = test_set['smoking']

In [ ]:
import optuna
from pytorch_tabnet.tab_model import TabNetClassifier
from sklearn.metrics import accuracy_score
import numpy as np

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

def objective(trial):
    
    n_d = trial.suggest_int("n_d", 8, 64, step=8)
    n_a = trial.suggest_int("n_a", 8, 64, step=8)
    n_steps = trial.suggest_int("n_steps", 3, 10)
    gamma = trial.suggest_float("gamma", 1.0, 2.0)
    lambda_sparse = trial.suggest_float("lambda_sparse", 1e-6, 1e-3, log=True)
    lr = trial.suggest_float("lr", 1e-4, 1e-2, log=True)
    max_epochs = trial.suggest_int("max_epochs", 30, 100)

    scores = []

    for fold, (train_idx, val_idx) in enumerate(skf.split(X, y)):
        X_train_fold, X_val_fold = X.iloc[train_idx], X.iloc[val_idx]
        y_train_fold, y_val_fold = y.iloc[train_idx], y.iloc[val_idx]   


        scaler = StandardScaler()
        X_train_scaled = scaler.fit_transform(X_train_fold)
        X_val_scaled   = scaler.transform(X_val_fold)

        
        X_train_fold = X_train_scaled.astype(np.float32)
        X_val_fold   = X_val_scaled.astype(np.float32)
        y_train_fold = y_train_fold.values.astype(np.int64)
        y_val_fold   = y_val_fold.values.astype(np.int64)

        model = TabNetClassifier(
            n_d=n_d,
            n_a=n_a,
            n_steps=n_steps,
            gamma=gamma,
            lambda_sparse=lambda_sparse,
            optimizer_fn=torch.optim.Adam,
            optimizer_params=dict(lr=lr),
            verbose=0,
            seed=42
        )

        model.fit(
            X_train_fold, y_train_fold,
            eval_set=[(X_val_fold, y_val_fold)],
            eval_metric=['accuracy'],
            max_epochs=max_epochs,
            patience=10,
            batch_size=1024,
            virtual_batch_size=128
        )

        y_pred_val = model.predict(X_val_fold)
        acc = accuracy_score(y_val_fold, y_pred_val)
        
        scores.append(acc)


    return np.mean(scores)


sampler = optuna.samplers.TPESampler(seed=42)
study = optuna.create_study(direction="maximize", sampler=sampler)
study.optimize(objective, n_trials=20)

print("best_params:", study.best_params)
print("best_value:", study.best_value)


[I 2025-07-31 20:07:55,162] A new study created in memory with name: no-name-3006a420-d6e4-4859-bf38-95f516a33991


Stop training because you reached max_epochs = 34 with best_epoch = 33 and best_val_0_accuracy = 0.6605


/opt/anaconda3/lib/python3.12/site-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


Stop training because you reached max_epochs = 34 with best_epoch = 33 and best_val_0_accuracy = 0.6535


/opt/anaconda3/lib/python3.12/site-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


...


Early stopping occurred at epoch 23 with best_epoch = 13 and best_val_0_accuracy = 0.757


/opt/anaconda3/lib/python3.12/site-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)



Early stopping occurred at epoch 26 with best_epoch = 16 and best_val_0_accuracy = 0.7515


/opt/anaconda3/lib/python3.12/site-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)
[W 2025-07-31 20:10:03,415] Trial 1 failed with parameters: {'n_d': 56, 'n_a': 40, 'n_steps': 8, 'gamma': 1.0205844942958024, 'lambda_sparse': 0.0008123245085588687, 'lr': 0.004622589001020831, 'max_epochs': 45} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.12/site-packages/optuna/study/_optimize.py", line 201, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "/var/folders/0c/64_tgxx17czfrpvpt1mh277m0000gn/T/ipykernel_4718/3685791170.py", line 47, in objective
    model.fit(
  File "/opt/anaconda3/lib/python3.12/site-packages/pytorch_tabnet/abstract_model.py", line 258, in fit
    self._train_epoch(train_dataloader)
  File "/opt/anaconda3/lib/python3.12/site-packages/pytorch_tabnet/abstract_model.py", line

KeyboardInterrupt: 

In [ ]:

X_train_f, X_val_f, y_train_f, y_val_f = train_test_split(
    X, y, test_size=0.2, random_state=42
)


scaler = StandardScaler()
X_train_f_scaled = scaler.fit_transform(X_train_f)
X_val_f_scaled   = scaler.transform(X_val_f)
X_test_new_scaled  = scaler.transform(X_test_new)


X_train_scaled_np = X_train_f_scaled.astype(np.float32)
y_train_np = y_train_f.values.astype(np.int64)

X_val_scaled_np = X_val_f_scaled.astype(np.float32)
y_val_np = y_val_f.values.astype(np.int64)

X_test_new_scaled_np = X_test_new_scaled.astype(np.float32)
y_test_new_np = y_test_new.values.astype(np.int64)


tabnet_model = TabNetClassifier(
    optimizer_fn=torch.optim.Adam,
    optimizer_params=dict(lr=0.001),
    scheduler_params={"step_size":10, "gamma":0.9},
    scheduler_fn=torch.optim.lr_scheduler.StepLR,
    verbose=10,
    n_d=56, n_a=40, n_steps=8, gamma=1.267,
    lambda_sparse=5.976e-05,
    seed=42
)


tabnet_model.fit(
    X_train=X_train_scaled_np, y_train=y_train_np,
    eval_set=[(X_val_scaled_np, y_val_np)],
    eval_metric=['accuracy'],
    patience=10,
    batch_size=1024,
    virtual_batch_size=128,
    max_epochs=73,
    num_workers=4
)

y_train_pred = tabnet_model.predict(X_train_scaled_np)
train_acc = accuracy_score(y_train_np, y_train_pred)

print(f"TabNet Train Accuracy: {train_acc:.4f}")
print(classification_report(y_train_np, y_train_pred, digits=4))


y_pred = tabnet_model.predict(X_test_new_scaled_np)
acc = accuracy_score(y_test_new_np, y_pred)

print(f"\nTabNet Test Accuracy: {acc:.4f}")
print(classification_report(y_test_new_np, y_pred, digits=4))